In [ ]:
!pip install category_encoders

     |████████████████████████████████| 82 kB 183 kB/s 


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from category_encoders.hashing import HashingEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score

Mount google drive storage to the notebook

In [ ]:
from google.colab import drive
drive.mount('/content/drive') 

Mounted at /content/drive


Read the gnomad variant extraction raw data

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/MSc-project-lof/gnomad_variants_data_lof.csv")

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 680545 entries, 0 to 680544
Data columns (total 53 columns):
 #   Column                                     Non-Null Count   Dtype 
---  ------                                     --------------   ----- 
 0   Chromosome                                 680545 non-null  object
 1   Position                                   680545 non-null  object
 2   rsIDs                                      666083 non-null  object
 3   Reference                                  680545 non-null  object
 4   Alternate                                  680545 non-null  object
 5   Source                                     680545 non-null  object
 6   Filters - exomes                           570349 non-null  object
 7   Filters - genomes                          232297 non-null  object
 8   Transcript                                 680545 non-null  object
 9   HGVS Consequence                           680491 non-null  object
 10  Protein Consequence 

In [ ]:
df.head(10)

,Chromosome,Position,rsIDs,Reference,Alternate,Source,Filters - exomes,Filters - genomes,Transcript,HGVS Consequence,Protein Consequence,Transcript Consequence,VEP Annotation,ClinVar Clinical Significance,ClinVar Variation ID,Flags,Allele Count,Allele Number,Allele Frequency,Homozygote Count,Hemizygote Count,Allele Count African/African-American,Allele Number African/African-American,Homozygote Count African/African-American,Hemizygote Count African/African-American,Allele Count Latino/Admixed American,Allele Number Latino/Admixed American,Homozygote Count Latino/Admixed American,Hemizygote Count Latino/Admixed American,Allele Count Ashkenazi Jewish,Allele Number Ashkenazi Jewish,Homozygote Count Ashkenazi Jewish,Hemizygote Count Ashkenazi Jewish,Allele Count East Asian,Allele Number East Asian,Homozygote Count East Asian,Hemizygote Count East Asian,Allele Count European (Finnish),Allele Number European (Finnish),Homozygote Count European (Finnish),Hemizygote Count European (Finnish),Allele Count European (non-Finnish),Allele Number European (non-Finnish),Homozygote Count European (non-Finnish),Hemizygote Count European (non-Finnish),Allele Count Other,Allele Number Other,Homozygote Count Other,Hemizygote Count Other,Allele Count South Asian,Allele Number South Asian,Homozygote Count South Asian,Hemizygote Count South Asian
0,6,143816777,rs1395541901,C,G,gnomAD Genomes,NaN,PASS,ENST00000002165.6,c.*67G>C,NaN,c.*67G>C,3_prime_UTR_variant,NaN,NaN,NaN,1,31410,0.00003183699458771092,0,0,0,8718,0,0,0,848,0,0,0,290,0,0,0,1558,0,0,1,3476,0,0,0,15432,0,0,0,1088,0,0,0,0,0,0
1,6,143816782,rs1413425269,A,G,gnomAD Genomes,NaN,PASS,ENST00000002165.6,c.*62T>C,NaN,c.*62T>C,3_prime_UTR_variant,NaN,NaN,NaN,3,31406,0.00009552314844297268,0,0,1,8716,0,0,0,848,0,0,0,290,0,0,2,1560,0,0,0,3476,0,0,0,15430,0,0,0,1086,0,0,0,0,0,0
2,6,143816794,rs764222848,T,C,"gnomAD Exomes,gnomAD Genomes",PASS,PASS,ENST00000002165.6,c.*50A>G,NaN,c.*50A>G,3_prime_UTR_variant,NaN,NaN,NaN,9,267680,0.00003362223550508069,0,0,0,24624,0,0,8,31814,0,0,0,9176,0,0,1,19524,0,0,0,23960,0,0,0,123392,0,0,0,6778,0,0,0,28412,0,0
3,6,143816796,rs753845716,G,T,gnomAD Exomes,PASS,NaN,ENST00000002165.6,c.*48C>A,NaN,c.*48C>A,3_prime_UTR_variant,NaN,NaN,NaN,18,241214,0.00007462253434709429,0,0,0,16024,0,0,0,32652,0,0,0,9312,0,0,0,18124,0,0,0,20640,0,0,0,109374,0,0,0,5826,0,0,18,29262,0,0
4,6,143816802,rs1221211326,A,T,gnomAD Exomes,PASS,NaN,ENST00000002165.6,c.*42T>A,NaN,c.*42T>A,3_prime_UTR_variant,NaN,NaN,NaN,1,244028,0.0000040978904060189815,0,0,0,16084,0,0,0,33222,0,0,0,9572,0,0,1,18218,0,0,0,20956,0,0,0,110394,0,0,0,5906,0,0,0,29676,0,0
5,6,143816812,rs1282169426,A,T,gnomAD Exomes,PASS,NaN,ENST00000002165.6,c.*32T>A,NaN,c.*32T>A,3_prime_UTR_variant,NaN,NaN,NaN,1,247698,0.00000403717430096327,0,0,0,16194,0,0,1,33988,0,0,0,9880,0,0,0,18320,0,0,0,21274,0,0,0,111830,0,0,0,6012,0,0,0,30200,0,0
6,6,143816813,rs761751116,T,A,gnomAD Exomes,PASS,NaN,ENST00000002165.6,c.*31A>T,NaN,c.*31A>T,3_prime_UTR_variant,NaN,NaN,NaN,2,247852,0.000008069331697948776,0,0,0,16190,0,0,0,34018,0,0,0,9892,0,0,2,18318,0,0,0,21312,0,0,0,111886,0,0,0,6014,0,0,0,30222,0,0
7,6,143816822,rs765109071,G,T,"gnomAD Exomes,gnomAD Genomes",PASS,PASS,ENST00000002165.6,c.*22C>A,NaN,c.*22C>A,3_prime_UTR_variant,NaN,NaN,NaN,4,280098,0.000014280716035102,0,0,0,24920,0,0,0,34966,0,0,0,10224,0,0,0,19910,0,0,0,24876,0,0,3,127684,0,0,1,7130,0,0,0,30388,0,0
8,6,143816824,rs1252379980,A,G,gnomAD Exomes,PASS,NaN,ENST00000002165.6,c.*20T>C,NaN,c.*20T>C,3_prime_UTR_variant,NaN,NaN,NaN,1,249158,0.000004013517527031041,0,0,0,16228,0,0,1,34196,0,0,0,9980,0,0,0,18364,0,0,0,21450,0,0,0,112456,0,0,0,6056,0,0,0,30428,0,0
9,6,143816825,rs750214725,T,G,"gnomAD Exomes,gnomAD Genomes",PASS,PASS,ENST00000002165.6,c.*19A>C,NaN,c.*19A>C,3_prime_UTR_variant,NaN,NaN,NaN,4,280538,0.000014258317946231883,0,0,0,24938,0,0,0,35026,0,0,0,10264,0,0,0,19918,0,0,0,24926,0,0,3,127892,0,0,1,7142,0,0,0,30432,0,0


Remove repeated header rows. Also note, ClinVar Clinical Significance is available for only 11980 records.

In [ ]:
df = df[df["Chromosome"]!="Chromosome"]
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 678654 entries, 0 to 680544
Data columns (total 53 columns):
 #   Column                                     Non-Null Count   Dtype 
---  ------                                     --------------   ----- 
 0   Chromosome                                 678654 non-null  object
 1   Position                                   678654 non-null  object
 2   rsIDs                                      664192 non-null  object
 3   Reference                                  678654 non-null  object
 4   Alternate                                  678654 non-null  object
 5   Source                                     678654 non-null  object
 6   Filters - exomes                           568458 non-null  object
 7   Filters - genomes                          230406 non-null  object
 8   Transcript                                 678654 non-null  object
 9   HGVS Consequence                           678600 non-null  object
 10  Protein Consequence 

Clinical significance frquencies

In [ ]:
df["ClinVar Clinical Significance"].value_counts()

Likely benign                                                3768
Benign                                                       2329
Uncertain significance                                       2241
Pathogenic                                                   1178
Conflicting interpretations of pathogenicity                 1005
Likely pathogenic                                             529
Benign/Likely benign                                          439
Pathogenic/Likely pathogenic                                  313
not provided                                                   86
drug response                                                  59
Affects                                                         9
association                                                     7
risk factor                                                     6
Likely benign, other                                            2
Likely benign, drug response, other                             2
Pathogenic

Keep only three types of significances, only 2020 records remain after filtering.

In [ ]:
df["ClinVar Clinical Significance"].value_counts()\
[["Pathogenic","Likely pathogenic","Pathogenic/Likely pathogenic"]]

Pathogenic                      1178
Likely pathogenic                529
Pathogenic/Likely pathogenic     313
Name: ClinVar Clinical Significance, dtype: int64

In [ ]:
df

,Chromosome,Position,rsIDs,Reference,Alternate,Source,Filters - exomes,Filters - genomes,Transcript,HGVS Consequence,Protein Consequence,Transcript Consequence,VEP Annotation,ClinVar Clinical Significance,ClinVar Variation ID,Flags,Allele Count,Allele Number,Allele Frequency,Homozygote Count,Hemizygote Count,Allele Count African/African-American,Allele Number African/African-American,Homozygote Count African/African-American,Hemizygote Count African/African-American,Allele Count Latino/Admixed American,Allele Number Latino/Admixed American,Homozygote Count Latino/Admixed American,Hemizygote Count Latino/Admixed American,Allele Count Ashkenazi Jewish,Allele Number Ashkenazi Jewish,Homozygote Count Ashkenazi Jewish,Hemizygote Count Ashkenazi Jewish,Allele Count East Asian,Allele Number East Asian,Homozygote Count East Asian,Hemizygote Count East Asian,Allele Count European (Finnish),Allele Number European (Finnish),Homozygote Count European (Finnish),Hemizygote Count European (Finnish),Allele Count European (non-Finnish),Allele Number European (non-Finnish),Homozygote Count European (non-Finnish),Hemizygote Count European (non-Finnish),Allele Count Other,Allele Number Other,Homozygote Count Other,Hemizygote Count Other,Allele Count South Asian,Allele Number South Asian,Homozygote Count South Asian,Hemizygote Count South Asian
1608,17,56348226,rs35897051,T,G,"gnomAD Exomes,gnomAD Genomes",PASS,PASS,ENST00000225275.3,c.2031-2A>C,NaN,c.2031-2A>C,splice_acceptor_variant,Pathogenic/Likely pathogenic,3632,NaN,1260,282532,0.004459671824784449,2,0,24,24940,0,0,106,35436,0,0,130,10360,0,0,1,19952,0,0,42,25116,0,0,936,128900,2,0,18,7212,0,0,3,30616,0,0
1711,17,56350827,rs963955387,GGGGTTGGGTTCCAT,G,"gnomAD Exomes,gnomAD Genomes",PASS,PASS,ENST00000225275.3,p.Met519ProfsTer21,p.Met519ProfsTer21,c.1555_1568delATGGAACCCAACCC,frameshift_variant,Pathogenic/Likely pathogenic,3629,NaN,221,282874,0.0007812665709821334,0,0,4,24964,0,0,13,35440,0,0,11,10370,0,0,0,19954,0,0,0,25124,0,0,189,129180,0,0,4,7226,0,0,0,30616,0,0
1754,17,56352986,rs762933005,TG,T,gnomAD Exomes,PASS,NaN,ENST00000225275.3,p.Thr428GlnfsTer6,p.Thr428GlnfsTer6,c.1281delC,frameshift_variant,Likely pathogenic,452644,NaN,3,251426,0.000011931940213024906,0,0,0,16252,0,0,0,34588,0,0,0,10078,0,0,0,18394,0,0,0,21624,0,0,3,113738,0,0,0,6136,0,0,0,30616,0,0
1875,17,56356732,rs778013714,C,A,"gnomAD Exomes,gnomAD Genomes",PASS,PASS,ENST00000225275.3,p.Glu202Ter,p.Glu202Ter,c.604G>T,stop_gained,Likely pathogenic,489145,NaN,32,246626,0.00012975112113078102,0,0,0,21330,0,0,0,31998,0,0,0,9570,0,0,1,17686,0,0,0,21656,0,0,31,110052,0,0,0,6406,0,0,0,27928,0,0
3966,17,48156838,rs200810866,C,T,gnomAD Exomes,PASS,NaN,ENST00000007722.7,p.Arg875Ter,p.Arg875Ter,c.2623C>T,stop_gained,Likely pathogenic,817532,NaN,3,215956,0.000013891718683435515,0,0,1,15578,0,0,0,27554,0,0,0,7154,0,0,0,16708,0,0,0,19838,0,0,2,98612,0,0,0,5136,0,0,0,25376,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
663956,6,32803059,rs61736923,G,A,gnomAD Exomes,PASS,NaN,ENST00000452392.2,p.Arg273Ter,p.Arg273Ter,c.817C>T,stop_gained,Pathogenic,13727,NaN,1,246732,0.000004052980561905225,0,0,0,15150,0,0,1,34480,0,0,0,9974,0,0,0,18274,0,0,0,21636,0,0,0,110762,0,0,0,6078,0,0,0,30378,0,0
664060,6,32805792,rs1321880935,GGT,G,gnomAD Exomes,PASS,NaN,ENST00000452392.2,p.Thr73ProfsTer93,p.Thr73ProfsTer93,c.217_218delAC,frameshift_variant,Likely pathogenic,626224,NaN,1,246008,0.0000040649084582615195,0,0,0,15066,0,0,0,34478,0,0,0,9946,0,0,0,18270,0,0,0,21594,0,0,1,110218,0,0,0,6058,0,0,0,30378,0,0
675271,16,89986560,rs143395134,C,G,gnomAD Exomes,PASS,NaN,ENST00000555147.1,p.Tyr298Ter,p.Tyr298Ter,c.894C>G,stop_gained,Likely pathogenic,548661,NaN,2,249030,0.000008031160904308718,0,0,0,15436,0,0,0,34522,0,0,0,10052,0,0,0,17970,0,0,0,21546,0,0,0,112854,0,0,1,6048,0,0,1,30602,0,0
6

Load constraints data, downloaded from gnomAD.

In [ ]:
df_metrics = pd.read_excel("/content/drive/MyDrive/MSc-project-lof/gnomad.v2.1.1.lof_metrics.by_transcript new.xlsx")

In [ ]:
df_metrics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80950 entries, 0 to 80949
Data columns (total 78 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   gene                80950 non-null  object 
 1   transcript          80950 non-null  object 
 2   canonical           80950 non-null  bool   
 3   obs_mis             80942 non-null  float64
 4   exp_mis             80942 non-null  float64
 5   oe_mis              80917 non-null  float64
 6   mu_mis              80942 non-null  float64
 7   possible_mis        80942 non-null  float64
 8   obs_mis_pphen       78998 non-null  float64
 9   exp_mis_pphen       78998 non-null  float64
 10  oe_mis_pphen        78937 non-null  float64
 11  possible_mis_pphen  78998 non-null  float64
 12  obs_syn             80941 non-null  float64
 13  exp_syn             80941 non-null  float64
 14  oe_syn              80914 non-null  float64
 15  mu_syn              80941 non-null  float64
 16  poss

Delete unwanted columns from metrics DF, keep only LOEUF constraint

In [ ]:
df_metrics = df_metrics[["transcript","oe_lof_upper"]]
df_metrics.head(10)

,transcript,oe_lof_upper
0,ENST00000263100,1.208
1,ENST00000600966,1.328
2,ENST00000282641,0.838
3,ENST00000373993,0.838
4,ENST00000373995,0.880
5,ENST00000373997,0.842
6,ENST00000374001,0.842
7,ENST00000395489,0.884
8,ENST00000395495,0.718
9,ENST00000414883,0.879


Main data frame has some additonal characters in transcript values. Remove extra chars after 15 chars and save in a new column so that it can be joined with contraints data frame.

In [ ]:
df["Transcript_j"] = df["Transcript"]

In [ ]:
df["Transcript_j"] = df["Transcript_j"].apply(lambda x: x[:15])

In [ ]:
df.head(10)

,Chromosome,Position,rsIDs,Reference,Alternate,Source,Filters - exomes,Filters - genomes,Transcript,HGVS Consequence,Protein Consequence,Transcript Consequence,VEP Annotation,ClinVar Clinical Significance,ClinVar Variation ID,Flags,Allele Count,Allele Number,Allele Frequency,Homozygote Count,Hemizygote Count,Allele Count African/African-American,Allele Number African/African-American,Homozygote Count African/African-American,Hemizygote Count African/African-American,Allele Count Latino/Admixed American,Allele Number Latino/Admixed American,Homozygote Count Latino/Admixed American,Hemizygote Count Latino/Admixed American,Allele Count Ashkenazi Jewish,Allele Number Ashkenazi Jewish,Homozygote Count Ashkenazi Jewish,Hemizygote Count Ashkenazi Jewish,Allele Count East Asian,Allele Number East Asian,Homozygote Count East Asian,Hemizygote Count East Asian,Allele Count European (Finnish),Allele Number European (Finnish),Homozygote Count European (Finnish),Hemizygote Count European (Finnish),Allele Count European (non-Finnish),Allele Number European (non-Finnish),Homozygote Count European (non-Finnish),Hemizygote Count European (non-Finnish),Allele Count Other,Allele Number Other,Homozygote Count Other,Hemizygote Count Other,Allele Count South Asian,Allele Number South Asian,Homozygote Count South Asian,Hemizygote Count South Asian,Transcript_j
1608,17,56348226,rs35897051,T,G,"gnomAD Exomes,gnomAD Genomes",PASS,PASS,ENST00000225275.3,c.2031-2A>C,NaN,c.2031-2A>C,splice_acceptor_variant,Pathogenic/Likely pathogenic,3632,NaN,1260,282532,0.004459671824784449,2,0,24,24940,0,0,106,35436,0,0,130,10360,0,0,1,19952,0,0,42,25116,0,0,936,128900,2,0,18,7212,0,0,3,30616,0,0,ENST00000225275
1711,17,56350827,rs963955387,GGGGTTGGGTTCCAT,G,"gnomAD Exomes,gnomAD Genomes",PASS,PASS,ENST00000225275.3,p.Met519ProfsTer21,p.Met519ProfsTer21,c.1555_1568delATGGAACCCAACCC,frameshift_variant,Pathogenic/Likely pathogenic,3629,NaN,221,282874,0.0007812665709821334,0,0,4,24964,0,0,13,35440,0,0,11,10370,0,0,0,19954,0,0,0,25124,0,0,189,129180,0,0,4,7226,0,0,0,30616,0,0,ENST00000225275
1754,17,56352986,rs762933005,TG,T,gnomAD Exomes,PASS,NaN,ENST00000225275.3,p.Thr428GlnfsTer6,p.Thr428GlnfsTer6,c.1281delC,frameshift_variant,Likely pathogenic,452644,NaN,3,251426,0.000011931940213024906,0,0,0,16252,0,0,0,34588,0,0,0,10078,0,0,0,18394,0,0,0,21624,0,0,3,113738,0,0,0,6136,0,0,0,30616,0,0,ENST00000225275
1875,17,56356732,rs778013714,C,A,"gnomAD Exomes,gnomAD Genomes",PASS,PASS,ENST00000225275.3,p.Glu202Ter,p.Glu202Ter,c.604G>T,stop_gained,Likely pathogenic,489145,NaN,32,246626,0.00012975112113078102,0,0,0,21330,0,0,0,31998,0,0,0,9570,0,0,1,17686,0,0,0,21656,0,0,31,110052,0,0,0,6406,0,0,0,27928,0,0,ENST00000225275
3966,17,48156838,rs200810866,C,T,gnomAD Exomes,PASS,NaN,ENST00000007722.7,p.Arg875Ter,p.Arg875Ter,c.2623C>T,stop_gained,Likely pathogenic,817532,NaN,3,215956,0.000013891718683435515,0,0,1,15578,0,0,0,27554,0,0,0,7154,0,0,0,16708,0,0,0,19838,0,0,2,98612,0,0,0,5136,0,0,0,25376,0,0,ENST00000007722
7082,17,11548020,rs368341716,A,G,"gnomAD Exomes,gnomAD Genomes",PASS,PASS,ENST00000262442.4,c.1970+4A>G,NaN,c.1970+4A>G,splice_region_variant,Pathogenic,617523,NaN,41,281126,0.00014584207792946936,0,0,5,24932,0,0,0,35098,0,0,0,10346,0,0,0,19886,0,0,0,24450,0,0,36,128848,0,0,0,7186,0,0,0,30380,0,0,ENST00000262442
7320,17,11572741,rs771595064,AG,A,gnomAD Exomes,PASS,NaN,ENST00000262442.4,p.Arg995LysfsTer5,p.Arg995LysfsTer5,c.2984delG,frameshift_variant,Pathogenic,1065498,NaN,6,251288,0.000023876985769316483,0,0,0,16256,0,0,5,34566,0,0,0,10072,0,0,0,18392,0,0,0,21634,0,0,0,113632,0,0,1,6130,0,0,0,30606,0,0,ENST00000262442
7354,17,11583072,rs768238895,AG,A,gnomAD Exomes,PASS,NaN,ENST00000262442.4,c.3354-1delG,NaN,c.3354-1delG,splice_acceptor_variant,Likely pathogenic,986968,NaN,2,251302,0.000007958551861903208,0,0,0,16254,0,0,0,34570,0,0,0,10078,0,0,0,18394,0,0,0,21600,0,0,2,113662,0,0,0,6138,0,0,0,30606,0,0,ENST00000262442
7431,17,11593373,rs1331883139,C,T,gnomAD Genomes,NaN,PA

Rename the transcript column in metrics data frame before joining.

In [ ]:
df_metrics.rename(columns={"transcript":"Transcript_j"},inplace=True)

In [ ]:
df_metrics.columns

Index(['Unnamed: 0', 'Transcript_j', 'oe_lof_upper'], dtype='object')

Peform join

In [ ]:
df = df.join(df_metrics.set_index("Transcript_j"),on="Transcript_j")
df.head(10)

,Chromosome,Position,rsIDs,Reference,Alternate,Source,Filters - exomes,Filters - genomes,Transcript,HGVS Consequence,Protein Consequence,Transcript Consequence,VEP Annotation,ClinVar Clinical Significance,ClinVar Variation ID,Flags,Allele Count,Allele Number,Allele Frequency,Homozygote Count,Hemizygote Count,Allele Count African/African-American,Allele Number African/African-American,Homozygote Count African/African-American,Hemizygote Count African/African-American,Allele Count Latino/Admixed American,Allele Number Latino/Admixed American,Homozygote Count Latino/Admixed American,Hemizygote Count Latino/Admixed American,Allele Count Ashkenazi Jewish,Allele Number Ashkenazi Jewish,Homozygote Count Ashkenazi Jewish,Hemizygote Count Ashkenazi Jewish,Allele Count East Asian,Allele Number East Asian,Homozygote Count East Asian,Hemizygote Count East Asian,Allele Count European (Finnish),Allele Number European (Finnish),Homozygote Count European (Finnish),Hemizygote Count European (Finnish),Allele Count European (non-Finnish),Allele Number European (non-Finnish),Homozygote Count European (non-Finnish),Hemizygote Count European (non-Finnish),Allele Count Other,Allele Number Other,Homozygote Count Other,Hemizygote Count Other,Allele Count South Asian,Allele Number South Asian,Homozygote Count South Asian,Hemizygote Count South Asian,Transcript_j,oe_lof_upper
1608,17,56348226,rs35897051,T,G,"gnomAD Exomes,gnomAD Genomes",PASS,PASS,ENST00000225275.3,c.2031-2A>C,NaN,c.2031-2A>C,splice_acceptor_variant,Pathogenic/Likely pathogenic,3632,NaN,1260,282532,0.004459671824784449,2,0,24,24940,0,0,106,35436,0,0,130,10360,0,0,1,19952,0,0,42,25116,0,0,936,128900,2,0,18,7212,0,0,3,30616,0,0,ENST00000225275,1.145
1711,17,56350827,rs963955387,GGGGTTGGGTTCCAT,G,"gnomAD Exomes,gnomAD Genomes",PASS,PASS,ENST00000225275.3,p.Met519ProfsTer21,p.Met519ProfsTer21,c.1555_1568delATGGAACCCAACCC,frameshift_variant,Pathogenic/Likely pathogenic,3629,NaN,221,282874,0.0007812665709821334,0,0,4,24964,0,0,13,35440,0,0,11,10370,0,0,0,19954,0,0,0,25124,0,0,189,129180,0,0,4,7226,0,0,0,30616,0,0,ENST00000225275,1.145
1754,17,56352986,rs762933005,TG,T,gnomAD Exomes,PASS,NaN,ENST00000225275.3,p.Thr428GlnfsTer6,p.Thr428GlnfsTer6,c.1281delC,frameshift_variant,Likely pathogenic,452644,NaN,3,251426,0.000011931940213024906,0,0,0,16252,0,0,0,34588,0,0,0,10078,0,0,0,18394,0,0,0,21624,0,0,3,113738,0,0,0,6136,0,0,0,30616,0,0,ENST00000225275,1.145
1875,17,56356732,rs778013714,C,A,"gnomAD Exomes,gnomAD Genomes",PASS,PASS,ENST00000225275.3,p.Glu202Ter,p.Glu202Ter,c.604G>T,stop_gained,Likely pathogenic,489145,NaN,32,246626,0.00012975112113078102,0,0,0,21330,0,0,0,31998,0,0,0,9570,0,0,1,17686,0,0,0,21656,0,0,31,110052,0,0,0,6406,0,0,0,27928,0,0,ENST00000225275,1.145
3966,17,48156838,rs200810866,C,T,gnomAD Exomes,PASS,NaN,ENST00000007722.7,p.Arg875Ter,p.Arg875Ter,c.2623C>T,stop_gained,Likely pathogenic,817532,NaN,3,215956,0.000013891718683435515,0,0,1,15578,0,0,0,27554,0,0,0,7154,0,0,0,16708,0,0,0,19838,0,0,2,98612,0,0,0,5136,0,0,0,25376,0,0,ENST00000007722,0.534
7082,17,11548020,rs368341716,A,G,"gnomAD Exomes,gnomAD Genomes",PASS,PASS,ENST00000262442.4,c.1970+4A>G,NaN,c.1970+4A>G,splice_region_variant,Pathogenic,617523,NaN,41,281126,0.00014584207792946936,0,0,5,24932,0,0,0,35098,0,0,0,10346,0,0,0,19886,0,0,0,24450,0,0,36,128848,0,0,0,7186,0,0,0,30380,0,0,ENST00000262442,0.722
7320,17,11572741,rs771595064,AG,A,gnomAD Exomes,PASS,NaN,ENST00000262442.4,p.Arg995LysfsTer5,p.Arg995LysfsTer5,c.2984delG,frameshift_variant,Pathogenic,1065498,NaN,6,251288,0.000023876985769316483,0,0,0,16256,0,0,5,34566,0,0,0,10072,0,0,0,18392,0,0,0,21634,0,0,0,113632,0,0,1,6130,0,0,0,30606,0,0,ENST00000262442,0.722
7354,17,11583072,rs768238895,AG,A,gnomAD Exomes,PASS,NaN,ENST00000262442.4,c.3354-1delG,NaN,c.3354-1delG,splice_acceptor_variant,Likely pathogenic,986968,NaN,2,251302,0.000007958551861903208,0,0,0,16254,0,0,0,34570,0,0,0,10078,0,0,0,18394,0,0,0,21600,0,0,2,113662,0,0,0,6138,0,0,0,30606,0,0,ENST00000262442,

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2020 entries, 1608 to 679977
Data columns (total 55 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   Chromosome                                 2020 non-null   object 
 1   Position                                   2020 non-null   object 
 2   rsIDs                                      1997 non-null   object 
 3   Reference                                  2020 non-null   object 
 4   Alternate                                  2020 non-null   object 
 5   Source                                     2020 non-null   object 
 6   Filters - exomes                           1890 non-null   object 
 7   Filters - genomes                          613 non-null    object 
 8   Transcript                                 2020 non-null   object 
 9   HGVS Consequence                           2020 non-null   object 
 10  Protein Consequence

In [ ]:
df

,Chromosome,Position,rsIDs,Reference,Alternate,Source,Filters - exomes,Filters - genomes,Transcript,HGVS Consequence,Protein Consequence,Transcript Consequence,VEP Annotation,ClinVar Clinical Significance,ClinVar Variation ID,Flags,Allele Count,Allele Number,Allele Frequency,Homozygote Count,Hemizygote Count,Allele Count African/African-American,Allele Number African/African-American,Homozygote Count African/African-American,Hemizygote Count African/African-American,Allele Count Latino/Admixed American,Allele Number Latino/Admixed American,Homozygote Count Latino/Admixed American,Hemizygote Count Latino/Admixed American,Allele Count Ashkenazi Jewish,Allele Number Ashkenazi Jewish,Homozygote Count Ashkenazi Jewish,Hemizygote Count Ashkenazi Jewish,Allele Count East Asian,Allele Number East Asian,Homozygote Count East Asian,Hemizygote Count East Asian,Allele Count European (Finnish),Allele Number European (Finnish),Homozygote Count European (Finnish),Hemizygote Count European (Finnish),Allele Count European (non-Finnish),Allele Number European (non-Finnish),Homozygote Count European (non-Finnish),Hemizygote Count European (non-Finnish),Allele Count Other,Allele Number Other,Homozygote Count Other,Hemizygote Count Other,Allele Count South Asian,Allele Number South Asian,Homozygote Count South Asian,Hemizygote Count South Asian,Transcript_j,oe_lof_upper,CADD key
1608,17,56348226,rs35897051,T,G,"gnomAD Exomes,gnomAD Genomes",PASS,PASS,ENST00000225275.3,c.2031-2A>C,NaN,c.2031-2A>C,splice_acceptor_variant,Pathogenic/Likely pathogenic,3632,NaN,1260,282532,0.004459671824784449,2,0,24,24940,0,0,106,35436,0,0,130,10360,0,0,1,19952,0,0,42,25116,0,0,936,128900,2,0,18,7212,0,0,3,30616,0,0,ENST00000225275,1.145,1756348226TG
1711,17,56350827,rs963955387,GGGGTTGGGTTCCAT,G,"gnomAD Exomes,gnomAD Genomes",PASS,PASS,ENST00000225275.3,p.Met519ProfsTer21,p.Met519ProfsTer21,c.1555_1568delATGGAACCCAACCC,frameshift_variant,Pathogenic/Likely pathogenic,3629,NaN,221,282874,0.0007812665709821334,0,0,4,24964,0,0,13,35440,0,0,11,10370,0,0,0,19954,0,0,0,25124,0,0,189,129180,0,0,4,7226,0,0,0,30616,0,0,ENST00000225275,1.145,1756350827GGGGTTGGGTTCCATG
1754,17,56352986,rs762933005,TG,T,gnomAD Exomes,PASS,NaN,ENST00000225275.3,p.Thr428GlnfsTer6,p.Thr428GlnfsTer6,c.1281delC,frameshift_variant,Likely pathogenic,452644,NaN,3,251426,0.000011931940213024906,0,0,0,16252,0,0,0,34588,0,0,0,10078,0,0,0,18394,0,0,0,21624,0,0,3,113738,0,0,0,6136,0,0,0,30616,0,0,ENST00000225275,1.145,1756352986TGT
1875,17,56356732,rs778013714,C,A,"gnomAD Exomes,gnomAD Genomes",PASS,PASS,ENST00000225275.3,p.Glu202Ter,p.Glu202Ter,c.604G>T,stop_gained,Likely pathogenic,489145,NaN,32,246626,0.00012975112113078102,0,0,0,21330,0,0,0,31998,0,0,0,9570,0,0,1,17686,0,0,0,21656,0,0,31,110052,0,0,0,6406,0,0,0,27928,0,0,ENST00000225275,1.145,1756356732CA
3966,17,48156838,rs200810866,C,T,gnomAD Exomes,PASS,NaN,ENST00000007722.7,p.Arg875Ter,p.Arg875Ter,c.2623C>T,stop_gained,Likely pathogenic,817532,NaN,3,215956,0.000013891718683435515,0,0,1,15578,0,0,0,27554,0,0,0,7154,0,0,0,16708,0,0,0,19838,0,0,2,98612,0,0,0,5136,0,0,0,25376,0,0,ENST00000007722,0.534,1748156838CT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
663956,6,32803059,rs61736923,G,A,gnomAD Exomes,PASS,NaN,ENST00000452392.2,p.Arg273Ter,p.Arg273Ter,c.817C>T,stop_gained,Pathogenic,13727,NaN,1,246732,0.000004052980561905225,0,0,0,15150,0,0,1,34480,0,0,0,9974,0,0,0,18274,0,0,0,21636,0,0,0,110762,0,0,0,6078,0,0,0,30378,0,0,ENST00000452392,0.806,632803059GA
664060,6,32805792,rs1321880935,GGT,G,gnomAD Exomes,PASS,NaN,ENST00000452392.2,p.Thr73ProfsTer93,p.Thr73ProfsTer93,c.217_218delAC,frameshift_variant,Likely pathogenic,626224,NaN,1,246008,0.0000040649084582615195,0,0,0,15066,0,0,0,34478,0,0,0,9946,0,0,0,18270,0,0,0,21594,0,0,1,110218,0,0,0,6058,0,0,0,30378,0,0,ENST000004523

Drop unwanted columns

In [ ]:
drop_list = ["Filters - exomes","Filters - genomes","ClinVar Variation ID","Flags","CADD key", \
             "Chromosome_cadd","Position_cadd","Reference_cadd","Alternate_cadd","RawScore","Position"]

In [ ]:
df.drop(drop_list,inplace=True,axis=1)
df

,Chromosome,rsIDs,Reference,Alternate,Source,Transcript,HGVS Consequence,Protein Consequence,Transcript Consequence,VEP Annotation,ClinVar Clinical Significance,Allele Count,Allele Number,Allele Frequency,Homozygote Count,Hemizygote Count,Allele Count African/African-American,Allele Number African/African-American,Homozygote Count African/African-American,Hemizygote Count African/African-American,Allele Count Latino/Admixed American,Allele Number Latino/Admixed American,Homozygote Count Latino/Admixed American,Hemizygote Count Latino/Admixed American,Allele Count Ashkenazi Jewish,Allele Number Ashkenazi Jewish,Homozygote Count Ashkenazi Jewish,Hemizygote Count Ashkenazi Jewish,Allele Count East Asian,Allele Number East Asian,Homozygote Count East Asian,Hemizygote Count East Asian,Allele Count European (Finnish),Allele Number European (Finnish),Homozygote Count European (Finnish),Hemizygote Count European (Finnish),Allele Count European (non-Finnish),Allele Number European (non-Finnish),Homozygote Count European (non-Finnish),Hemizygote Count European (non-Finnish),Allele Count Other,Allele Number Other,Homozygote Count Other,Hemizygote Count Other,Allele Count South Asian,Allele Number South Asian,Homozygote Count South Asian,Hemizygote Count South Asian,Transcript_j,oe_lof_upper,PHRED
1608,17,rs35897051,T,G,"gnomAD Exomes,gnomAD Genomes",ENST00000225275.3,c.2031-2A>C,NaN,c.2031-2A>C,splice_acceptor_variant,Pathogenic/Likely pathogenic,1260,282532,0.004459671824784449,2,0,24,24940,0,0,106,35436,0,0,130,10360,0,0,1,19952,0,0,42,25116,0,0,936,128900,2,0,18,7212,0,0,3,30616,0,0,ENST00000225275,1.145,NaN
1711,17,rs963955387,GGGGTTGGGTTCCAT,G,"gnomAD Exomes,gnomAD Genomes",ENST00000225275.3,p.Met519ProfsTer21,p.Met519ProfsTer21,c.1555_1568delATGGAACCCAACCC,frameshift_variant,Pathogenic/Likely pathogenic,221,282874,0.0007812665709821334,0,0,4,24964,0,0,13,35440,0,0,11,10370,0,0,0,19954,0,0,0,25124,0,0,189,129180,0,0,4,7226,0,0,0,30616,0,0,ENST00000225275,1.145,NaN
1754,17,rs762933005,TG,T,gnomAD Exomes,ENST00000225275.3,p.Thr428GlnfsTer6,p.Thr428GlnfsTer6,c.1281delC,frameshift_variant,Likely pathogenic,3,251426,0.000011931940213024906,0,0,0,16252,0,0,0,34588,0,0,0,10078,0,0,0,18394,0,0,0,21624,0,0,3,113738,0,0,0,6136,0,0,0,30616,0,0,ENST00000225275,1.145,NaN
1875,17,rs778013714,C,A,"gnomAD Exomes,gnomAD Genomes",ENST00000225275.3,p.Glu202Ter,p.Glu202Ter,c.604G>T,stop_gained,Likely pathogenic,32,246626,0.00012975112113078102,0,0,0,21330,0,0,0,31998,0,0,0,9570,0,0,1,17686,0,0,0,21656,0,0,31,110052,0,0,0,6406,0,0,0,27928,0,0,ENST00000225275,1.145,NaN
3966,17,rs200810866,C,T,gnomAD Exomes,ENST00000007722.7,p.Arg875Ter,p.Arg875Ter,c.2623C>T,stop_gained,Likely pathogenic,3,215956,0.000013891718683435515,0,0,1,15578,0,0,0,27554,0,0,0,7154,0,0,0,16708,0,0,0,19838,0,0,2,98612,0,0,0,5136,0,0,0,25376,0,0,ENST00000007722,0.534,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
663956,6,rs61736923,G,A,gnomAD Exomes,ENST00000452392.2,p.Arg273Ter,p.Arg273Ter,c.817C>T,stop_gained,Pathogenic,1,246732,0.000004052980561905225,0,0,0,15150,0,0,1,34480,0,0,0,9974,0,0,0,18274,0,0,0,21636,0,0,0,110762,0,0,0,6078,0,0,0,30378,0,0,ENST00000452392,0.806,NaN
664060,6,rs1321880935,GGT,G,gnomAD Exomes,ENST00000452392.2,p.Thr73ProfsTer93,p.Thr73ProfsTer93,c.217_218delAC,frameshift_variant,Likely pathogenic,1,246008,0.0000040649084582615195,0,0,0,15066,0,0,0,34478,0,0,0,9946,0,0,0,18270,0,0,0,21594,0,0,1,110218,0,0,0,6058,0,0,0,30378,0,0,ENST00000452392,0.806,NaN
675271,16,rs143395134,C,G,gnomAD Exomes,ENST00000555147.1,p.Tyr298Ter,p.Tyr298Ter,c.894C>G,stop_gained,Likely pathogenic,2,249030,0.000008031160904308718,0,0,0,15436,0,0,0,34522,0,0,0,10052,0,0,0,17970,0,0,0,21546,0,0,0,112854,0,0,1,6048,0,0,1,30602,0,0,ENST00000555147,1.967,NaN
675562,1,rs762721434,T,C,gnomAD Exomes,ENST00000557284.2,c.675+2T>C,Na

Split main DF into X and Y, features and lables. Drop unwanted columns.

In [ ]:
df_X = df
df_y = df["ClinVar Clinical Significance"]

In [ ]:
drop_X_train = ["rsIDs","Source","ClinVar Clinical Significance","Transcript_j"]

In [ ]:
df_X.drop(drop_X_train,axis=1,inplace=True)

In [ ]:
df_X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2020 entries, 1608 to 679977
Data columns (total 47 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   Chromosome                                 2020 non-null   object 
 1   Reference                                  2020 non-null   object 
 2   Alternate                                  2020 non-null   object 
 3   Transcript                                 2020 non-null   object 
 4   HGVS Consequence                           2020 non-null   object 
 5   Protein Consequence                        1562 non-null   object 
 6   Transcript Consequence                     2020 non-null   object 
 7   VEP Annotation                             2020 non-null   object 
 8   Allele Count                               2020 non-null   object 
 9   Allele Number                              2020 non-null   object 
 10  Allele Frequency   

Update data type of the columns.

In [ ]:
dict_astype = {"Chromosome":"category",  #"Position":"int32","Reference":"object",\
               "Alternate":"object","Transcript":"object","HGVS Consequence":"object",\
               "Protein Consequence":"object","Transcript Consequence":"object","VEP Annotation":"category",\
               "Allele Count":"int32","Allele Number":"int32","Allele Frequency":"float64","Homozygote Count":"int32",\
               "Hemizygote Count":"int32","Allele Count African/African-American":"int32",\
               "Allele Number African/African-American":"int32",\
               "Homozygote Count African/African-American":"int32","Hemizygote Count African/African-American":"int32",\
               "Allele Count Latino/Admixed American":"int32","Allele Number Latino/Admixed American":"int32",\
               "Homozygote Count Latino/Admixed American":"int32",\
               "Hemizygote Count Latino/Admixed American":"int32","Allele Count Ashkenazi Jewish":"int32",\
               "Allele Number Ashkenazi Jewish":"int32","Homozygote Count Ashkenazi Jewish":"int32",\
               "Hemizygote Count Ashkenazi Jewish":"int32","Allele Count East Asian":"int32",\
               "Allele Number East Asian":"int32","Homozygote Count East Asian":"int32",\
               "Hemizygote Count East Asian":"int32","Allele Count European (Finnish)":"int32",\
               "Allele Number European (Finnish)":"int32","Homozygote Count European (Finnish)":"int32",\
               "Hemizygote Count European (Finnish)":"int32","Allele Count European (non-Finnish)":"int32",\
               "Allele Number European (non-Finnish)":"int32","Homozygote Count European (non-Finnish)":"int32",\
               "Hemizygote Count European (non-Finnish)":"int32","Allele Count Other":"int32",\
               "Allele Number Other":"int32","Homozygote Count Other":"int32","Hemizygote Count Other":"int32",\
               "Allele Count South Asian":"int32","Allele Number South Asian":"int32",\
               "Homozygote Count South Asian":"int32","Hemizygote Count South Asian":"int32",\
               "oe_lof_upper":"float64","PHRED":"float64"}
dict_astype               

{'Allele Count': 'int32',
 'Allele Count African/African-American': 'int32',
 'Allele Count Ashkenazi Jewish': 'int32',
 'Allele Count East Asian': 'int32',
 'Allele Count European (Finnish)': 'int32',
 'Allele Count European (non-Finnish)': 'int32',
 'Allele Count Latino/Admixed American': 'int32',
 'Allele Count Other': 'int32',
 'Allele Count South Asian': 'int32',
 'Allele Frequency': 'float64',
 'Allele Number': 'int32',
 'Allele Number African/African-American': 'int32',
 'Allele Number Ashkenazi Jewish': 'int32',
 'Allele Number East Asian': 'int32',
 'Allele Number European (Finnish)': 'int32',
 'Allele Number European (non-Finnish)': 'int32',
 'Allele Number Latino/Admixed American': 'int32',
 'Allele Number Other': 'int32',
 'Allele Number South Asian': 'int32',
 'Alternate': 'object',
 'Chromosome': 'category',
 'HGVS Consequence': 'object',
 'Hemizygote Count': 'int32',
 'Hemizygote Count African/African-American': 'int32',
 'Hemizygote Count Ashkenazi Jewish': 'int32',
 'H

In [ ]:
df_X = df_X.astype(dict_astype)
df_X

,Chromosome,Reference,Alternate,Transcript,HGVS Consequence,Protein Consequence,Transcript Consequence,VEP Annotation,Allele Count,Allele Number,Allele Frequency,Homozygote Count,Hemizygote Count,Allele Count African/African-American,Allele Number African/African-American,Homozygote Count African/African-American,Hemizygote Count African/African-American,Allele Count Latino/Admixed American,Allele Number Latino/Admixed American,Homozygote Count Latino/Admixed American,Hemizygote Count Latino/Admixed American,Allele Count Ashkenazi Jewish,Allele Number Ashkenazi Jewish,Homozygote Count Ashkenazi Jewish,Hemizygote Count Ashkenazi Jewish,Allele Count East Asian,Allele Number East Asian,Homozygote Count East Asian,Hemizygote Count East Asian,Allele Count European (Finnish),Allele Number European (Finnish),Homozygote Count European (Finnish),Hemizygote Count European (Finnish),Allele Count European (non-Finnish),Allele Number European (non-Finnish),Homozygote Count European (non-Finnish),Hemizygote Count European (non-Finnish),Allele Count Other,Allele Number Other,Homozygote Count Other,Hemizygote Count Other,Allele Count South Asian,Allele Number South Asian,Homozygote Count South Asian,Hemizygote Count South Asian,oe_lof_upper,PHRED
1608,17,T,G,ENST00000225275.3,c.2031-2A>C,NaN,c.2031-2A>C,splice_acceptor_variant,1260,282532,0.004460,2,0,24,24940,0,0,106,35436,0,0,130,10360,0,0,1,19952,0,0,42,25116,0,0,936,128900,2,0,18,7212,0,0,3,30616,0,0,1.145,NaN
1711,17,GGGGTTGGGTTCCAT,G,ENST00000225275.3,p.Met519ProfsTer21,p.Met519ProfsTer21,c.1555_1568delATGGAACCCAACCC,frameshift_variant,221,282874,0.000781,0,0,4,24964,0,0,13,35440,0,0,11,10370,0,0,0,19954,0,0,0,25124,0,0,189,129180,0,0,4,7226,0,0,0,30616,0,0,1.145,NaN
1754,17,TG,T,ENST00000225275.3,p.Thr428GlnfsTer6,p.Thr428GlnfsTer6,c.1281delC,frameshift_variant,3,251426,0.000012,0,0,0,16252,0,0,0,34588,0,0,0,10078,0,0,0,18394,0,0,0,21624,0,0,3,113738,0,0,0,6136,0,0,0,30616,0,0,1.145,NaN
1875,17,C,A,ENST00000225275.3,p.Glu202Ter,p.Glu202Ter,c.604G>T,stop_gained,32,246626,0.000130,0,0,0,21330,0,0,0,31998,0,0,0,9570,0,0,1,17686,0,0,0,21656,0,0,31,110052,0,0,0,6406,0,0,0,27928,0,0,1.145,NaN
3966,17,C,T,ENST00000007722.7,p.Arg875Ter,p.Arg875Ter,c.2623C>T,stop_gained,3,215956,0.000014,0,0,1,15578,0,0,0,27554,0,0,0,7154,0,0,0,16708,0,0,0,19838,0,0,2,98612,0,0,0,5136,0,0,0,25376,0,0,0.534,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
663956,6,G,A,ENST00000452392.2,p.Arg273Ter,p.Arg273Ter,c.817C>T,stop_gained,1,246732,0.000004,0,0,0,15150,0,0,1,34480,0,0,0,9974,0,0,0,18274,0,0,0,21636,0,0,0,110762,0,0,0,6078,0,0,0,30378,0,0,0.806,NaN
664060,6,GGT,G,ENST00000452392.2,p.Thr73ProfsTer93,p.Thr73ProfsTer93,c.217_218delAC,frameshift_variant,1,246008,0.000004,0,0,0,15066,0,0,0,34478,0,0,0,9946,0,0,0,18270,0,0,0,21594,0,0,1,110218,0,0,0,6058,0,0,0,30378,0,0,0.806,NaN
675271,16,C,G,ENST00000555147.1,p.Tyr298Ter,p.Tyr298Ter,c.894C>G,stop_gained,2,249030,0.000008,0,0,0,15436,0,0,0,34522,0,0,0,10052,0,0,0,17970,0,0,0,21546,0,0,0,112854,0,0,1,6048,0,0,1,30602,0,0,1.967,NaN
675562,1,T,C,ENST00000557284.2,c.675+2T>C,NaN,c.675+2T>C,splice_donor_variant,1,243416,0.000004,0,0,0,15894,0,0,0,32210,0,0,0,9864,0,0,0,17680,0,0,0,21546,0,0,1,111004,0,0,0,5856,0,0,0,29362,0,0,1.425,NaN


In [ ]:
df_X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2020 entries, 1608 to 679977
Data columns (total 47 columns):
 #   Column                                     Non-Null Count  Dtype   
---  ------                                     --------------  -----   
 0   Chromosome                                 2020 non-null   category
 1   Reference                                  2020 non-null   object  
 2   Alternate                                  2020 non-null   object  
 3   Transcript                                 2020 non-null   object  
 4   HGVS Consequence                           2020 non-null   object  
 5   Protein Consequence                        1562 non-null   object  
 6   Transcript Consequence                     2020 non-null   object  
 7   VEP Annotation                             2020 non-null   category
 8   Allele Count                               2020 non-null   int32   
 9   Allele Number                              2020 non-null   int32   
 10  Allele 

Save processed data.

In [ ]:
df_X.to_csv("/content/drive/MyDrive/MSc-project-lof/df_X_iter2_v1.csv",index=False)

In [ ]:
df_y.to_csv("/content/drive/MyDrive/MSc-project-lof/df_y_iter2_v1.csv",index=False)